# Test opening and reading from the Icechunk Store

## Step 0: Imports

In [24]:
import icechunk
import xarray as xr
import hvplot.xarray
import zarr

zarr.config.set({
    'async.concurrency': 100,
    'threading.max_workers': None
})

## Step 1: Setup Icechunk Store

In [7]:
# When opening the repo, use None for anonymous/public access
credentials = icechunk.containers_credentials({
    "gs://ismip6/": None
})
config = icechunk.RepositoryConfig.default()

icechunk_storage = icechunk.gcs_storage(
    bucket="ismip6-icechunk",
    prefix="12-07-2025",
    from_env=True
)
repo = icechunk.Repository.open(icechunk_storage, config, authorize_virtual_chunk_access=credentials)
session = repo.readonly_session(branch="main")

## Step 3 Test Opening with Zarr

In [8]:
# opening using Zarr
root = zarr.open(session.store, mode='r')

In [18]:
root.tree(level=1)

/
├── CPOM_BISICLES
│   ├── expT71_08
│   ├── expT73_08
│   └── expTD58_08
├── NCAR_CISM
│   ├── ctrl_proj_open
│   ├── ctrl_proj_std
│   ├── exp01
│   ├── exp02
│   ├── exp03
│   ├── exp04
│   ├── exp05
│   ├── exp06
│   ├── exp07
│   ├── exp08
│   ├── exp09
│   ├── exp10
│   ├── exp13
│   └── expA1
└── VUB_AISMPALEO
    ├── ctrl_proj_std
    ├── exp05
    ├── exp06
    ├── exp07
    ├── exp08
    ├── exp09
    ├── exp10
    ├── expA5
    ├── expA6
    ├── expA7
    ├── expB6
    ├── expB7
    ├── expB8
    └── hist_std

### Step 3a: Test reading data

In [20]:
# Get the array
prefix = '/VUB_AISMPALEO/ctrl_proj_std/base'
time_arr, x_arr, y_arr = root[f'{prefix}/time'], root[f'{prefix}/x'], root[f'{prefix}/y']

# Check the array's attributes
print(time_arr[0:10])
print(x_arr[0:10])
print(y_arr[0:10])

# Get the array
base_arr = root['/VUB_AISMPALEO/ctrl_proj_std/base/base']

# Check the array's attributes
print(base_arr[0,0:10,0])

[5478.6328125  5843.875      6209.1171875  6574.359375   6939.6015625
 7304.84423611 7670.08642361 8035.32861111 8400.5703125  8765.8125    ]
[-3040000. -3032000. -3024000. -3016000. -3008000. -3000000. -2992000.
 -2984000. -2976000. -2968000.]
[-3040000. -3032000. -3024000. -3016000. -3008000. -3000000. -2992000.
 -2984000. -2976000. -2968000.]
[-9.e+33 -9.e+33 -9.e+33 -9.e+33 -9.e+33 -9.e+33 -9.e+33 -9.e+33 -9.e+33
 -9.e+33]


## Step 4: Test Open + Plot with `xarray.open_datatree`

Note: It is not recommended to do this with the entire datatree.

In [11]:
dt = xr.open_datatree(
    session.store,
    engine="zarr",
    consolidated=False,
    group='CPOM_BISICLES/expT71_08'
)

/home/jupyter/conda_env/python313/lib/python3.13/site-packages/zarr/codecs/numcodecs/_codecs.py:141: ZarrUserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [12]:
dt

<xarray.DataTree>
Group: /
├── Group: /lithk
│       Dimensions:  (time: 90, x: 761, y: 761)
│       Coordinates:
│         * time     (time) float64 720B 365.2 730.5 1.096e+03 ... 3.251e+04 3.287e+04
│         * x        (x) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
│         * y        (y) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
│           lat      (x, y) float64 5MB ...
│           lon      (x, y) float64 5MB ...
│       Data variables:
│           lithk    (time, x, y) float64 417MB ...
│       Attributes:
│           info:      Model CPOM BISICLES
│           history:   bisiclesISMIPVarExtractorV2.py
│           meshName:  6080x6080km_8km_Antarctic_stereo
├── Group: /limnsw
│       Dimensions:  (time: 91)
│       Coordinates:
│         * time     (time) float64 728B 2.318e-310 2.318e-310 ... 1.23e-296 2.592e-165
│       Data variables:
│           limnsw   (time) float64 728B ...
│       Attributes:
│           info:     Model CPOM BISICLES
├── Group: /tendlibmassbffl
│       Dimensions:          (time: 91)
│       Coordinates:
│         * time             (time) float64 728B 365.2 730.5 ... 3.287e+04 3.324e+04
│       Data variables:
│           tendlibmassbffl  (time) float64 728B ...
│       Attributes:
│           info:     Model CPOM BISICLES
...
├── Group: /base
│       Dimensions:  (time: 90, x: 761, y: 761)
│       Coordinates:
│         * time     (time) float64 720B 365.2 730.5 1.096e+03 ... 3.251e+04 3.287e+04
│         * x        (x) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
│         * y        (y) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
│           lon      (x, y) float64 5MB ...
│           lat      (x, y) float64 5MB ...
│       Data variables:
│           base     (time, x, y) float64 417MB ...
│       Attributes:
│           info:      Model CPOM BISICLES
│           history:   bisiclesISMIPVarExtractorV2.py
│           meshName:  6080x6080km_8km_Antarctic_stereo
├── Group: /tendlicalvf
│       Dimensions:      (time: 91)
│       Coordinates:
│         * time         (time) float64 728B 2.318e-310 2.318e-310 ... 2.592e-165
│       Data variables:
│           tendlicalvf  (time) float64 728B ...
│       Attributes:
│           info:     Model CPOM BISICLES
└── Group: /licalvf
        Dimensions:  (time: 91, x: 761, y: 761)
        Coordinates:
          * time     (time) float64 728B 365.2 730.5 1.096e+03 ... 3.287e+04 3.324e+04
          * x        (x) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
          * y        (y) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
            lon      (x, y) float64 5MB ...
            lat      (x, y) float64 5MB ...
        Data variables:
            licalvf  (time, x, y) float64 422MB ...
        Attributes:
            info:      Model CPOM BISICLES
            history:   bisiclesISMIPVarExtractorV2.py
            meshName:  6080x6080km_8km_Antarctic_stereo

In [21]:
base_array = dt['base'].base
base_array

<xarray.DataArray 'base' (time: 90, x: 761, y: 761)> Size: 417MB
[52120890 values with dtype=float64]
Coordinates:
  * time     (time) float64 720B 365.2 730.5 1.096e+03 ... 3.251e+04 3.287e+04
  * x        (x) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * y        (y) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
    lon      (x, y) float64 5MB ...
    lat      (x, y) float64 5MB ...
Attributes:
    units:          m
    standard_name:  base_altitude

In [22]:
vmin, vmax = base_array.quantile([0.01, 0.99]).values
vmin, vmax

(np.float64(-1125.118165164284), np.float64(1415.013436026001))

In [25]:
base_array.hvplot.image(
    x='x', 
    y='y', 
    clim=(vmin, vmax), 
    cmap='RdBu',
    groupby='time',
    widget_location='bottom',
    aspect='equal',
    title=(" ").join(base_array.attrs['standard_name'].split("_"))
)

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom')
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(name='time', options={'365.24': np.float64(365....}, value=np.float64(365.24))